In [1]:
import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import pickle

from src.DQNAgent import Agent

from collections import deque

In [ ]:
train_episodes = 20000     

#Environment parameters
state_size = [84, 84, 4]
action_size = 4
stack_size = 4

#DQN parameters
learning_rate = 1e-3
gamma = 0.95                   
update_every = int(1e4)
tau = 1e-3

#Exploration parameters
explore_stop = 5e-2            
decay_rate = 3e-6

#Memory parameters
memory_size = int(1e6)
batch_size = 32

In [ ]:
env = gym.make('Breakout-v0')

tf.reset_default_graph()
with tf.device("/device:GPU:0"):    
    agent = Agent(state_size, action_size, stack_size, memory_size, batch_size, explore_stop, decay_rate, gamma, learning_rate, tau, update_every)

save_file = 'model/DQN.ckpt'
saver = tf.train.Saver(max_to_keep=20000)

config = tf.ConfigProto()
config.allow_soft_placement = True
config.gpu_options.allow_growth = True
#with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    agent.set_session(sess)    
    agent.hard_update()
       
    scores = []
    start = time.time()
    for ep in range(train_episodes):
        state = env.reset()
        state = agent.stack_frames(state, True)
        agent.reset()

        score = 0
        while True: 
            env.render()
            action, ex_p = agent.act(state, True)
            next_state, reward, done, _ = env.step(action)
            next_state = agent.stack_frames(next_state, False)
            score += reward
                        
            agent.step(state, action, reward, next_state, done)
            
            if done:
                break
            
            state = next_state
            
        scores.append(score)

        print("Ep {}\tScore: {}\tAverage Scores: {:.3f}\tMax Score: {}\tMin Score: {}\tloss: {:.5f}\tex_p: {:.2f}\ttime: {:.2f}"
              .format(ep, score, np.mean(scores[-100:]), np.max(scores[-100:]), np.amin(scores[-100:]), np.mean(agent.loss), ex_p, time.time() - start))
        start = time.time()
        
        if ep > 0 and ep % 100 == 0:        
            saver.save(sess, save_file+'_'+str(ep))
            with open("E:/openai/breakout/" + str(ep), 'wb') as file:
                pickle.dump(agent.t, file)
            
        if np.mean(scores[-100:]) >= 400.0:
            saver.save(sess, save_file)
            print("Environment solved in {} episodes, Average Scores: {}".format(ep+1, np.mean(scores[-100:])))
            break
            
fig = plt.figure()
fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.xlabel("Episodes")
plt.ylabel("Scores")
plt.show()            

d:\anaconda3\envs\openai\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
d:\anaconda3\envs\openai\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
d:\anaconda3\envs\openai\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Ep 0	Score: 2.0	Average Scores: 2.000	Max Score: 2.0	Min Score: 2.0	loss: 0.10561	ex_p: 1.00	time: 19.80
Ep 1	Score: 0.0	Average Scores: 1.000	Max Score: 2.0	Min Score: 0.0	loss: 0.00484	ex_p: 1.00	time: 10.54
Ep 2	Score: 3.0	Average Scores: 1.667	Max Score: 3.0	Min Score: 0.0	loss: 0.00356	ex_p: 1.00	time: 23.85
Ep 3	Score: 2.0	Average Scores: 1.750	Max Score: 3.0	Min Score: 0.0	loss: 0.00445	ex_p: 1.00	time: 15.76
Ep 4	Score: 1.0	Average Scores: 1.600	Max Score: 3.0	Min Score: 0.0	loss: 0.00337	ex_p: 1.00	time: 14.24
Ep 5	Score: 0.0	Average Scores: 1.333	Max Score: 3.0	Min Score: 0.0	loss: 0.00200	ex_p: 1.00	time: 10.62
Ep 6	Score: 7.0	Average Scores: 2.143	Max Score: 7.0	Min Score: 0.0	loss: 0.00269	ex_p: 0.99	time: 27.42
Ep 7	Score: 0.0	Average Scores: 1.875	Max Score: 7.0	Min Score: 0.0	loss: 0.00722	ex_p: 0.99	time: 10.96
Ep 8	Score: 1.0	Average Scores: 1.778	Max Score: 7.0	Min Score: 0.0	loss: 0.00435	ex_p: 0.99	time: 15.38
Ep 9	Score: 3.0	Average Scores: 1.900	Max Score: 7.0	Mi

Ep 78	Score: 2.0	Average Scores: 1.544	Max Score: 7.0	Min Score: 0.0	loss: 0.00155	ex_p: 0.94	time: 20.80
Ep 79	Score: 0.0	Average Scores: 1.525	Max Score: 7.0	Min Score: 0.0	loss: 0.00114	ex_p: 0.94	time: 13.39
Ep 80	Score: 1.0	Average Scores: 1.519	Max Score: 7.0	Min Score: 0.0	loss: 0.00105	ex_p: 0.94	time: 15.41
Ep 81	Score: 0.0	Average Scores: 1.500	Max Score: 7.0	Min Score: 0.0	loss: 0.00089	ex_p: 0.94	time: 12.83
Ep 82	Score: 0.0	Average Scores: 1.482	Max Score: 7.0	Min Score: 0.0	loss: 0.00074	ex_p: 0.94	time: 15.07
Ep 83	Score: 4.0	Average Scores: 1.512	Max Score: 7.0	Min Score: 0.0	loss: 0.00095	ex_p: 0.94	time: 29.11
Ep 84	Score: 1.0	Average Scores: 1.506	Max Score: 7.0	Min Score: 0.0	loss: 0.00069	ex_p: 0.94	time: 17.38
Ep 85	Score: 0.0	Average Scores: 1.488	Max Score: 7.0	Min Score: 0.0	loss: 0.00075	ex_p: 0.94	time: 13.58
Ep 86	Score: 0.0	Average Scores: 1.471	Max Score: 7.0	Min Score: 0.0	loss: 0.00095	ex_p: 0.94	time: 12.97
Ep 87	Score: 1.0	Average Scores: 1.466	Max Sco

Ep 155	Score: 1.0	Average Scores: 1.190	Max Score: 4.0	Min Score: 0.0	loss: 0.00125	ex_p: 0.89	time: 16.79
Ep 156	Score: 0.0	Average Scores: 1.190	Max Score: 4.0	Min Score: 0.0	loss: 0.00111	ex_p: 0.89	time: 13.20
Ep 157	Score: 1.0	Average Scores: 1.190	Max Score: 4.0	Min Score: 0.0	loss: 0.00117	ex_p: 0.89	time: 17.72
Ep 158	Score: 2.0	Average Scores: 1.200	Max Score: 4.0	Min Score: 0.0	loss: 0.00143	ex_p: 0.89	time: 22.81
Ep 159	Score: 0.0	Average Scores: 1.190	Max Score: 4.0	Min Score: 0.0	loss: 0.00155	ex_p: 0.89	time: 12.50
Ep 160	Score: 2.0	Average Scores: 1.190	Max Score: 4.0	Min Score: 0.0	loss: 0.00153	ex_p: 0.89	time: 19.29
Ep 161	Score: 4.0	Average Scores: 1.200	Max Score: 4.0	Min Score: 0.0	loss: 0.00235	ex_p: 0.89	time: 25.73
Ep 162	Score: 5.0	Average Scores: 1.250	Max Score: 5.0	Min Score: 0.0	loss: 0.00185	ex_p: 0.89	time: 29.00
Ep 163	Score: 2.0	Average Scores: 1.270	Max Score: 5.0	Min Score: 0.0	loss: 0.00122	ex_p: 0.89	time: 22.79
Ep 164	Score: 0.0	Average Scores: 1.2

Ep 232	Score: 4.0	Average Scores: 1.860	Max Score: 5.0	Min Score: 0.0	loss: 0.00132	ex_p: 0.84	time: 30.11
Ep 233	Score: 2.0	Average Scores: 1.880	Max Score: 5.0	Min Score: 0.0	loss: 0.00126	ex_p: 0.84	time: 20.94
Ep 234	Score: 1.0	Average Scores: 1.870	Max Score: 5.0	Min Score: 0.0	loss: 0.00142	ex_p: 0.84	time: 17.93
Ep 235	Score: 1.0	Average Scores: 1.870	Max Score: 5.0	Min Score: 0.0	loss: 0.00139	ex_p: 0.84	time: 19.52
Ep 236	Score: 1.0	Average Scores: 1.840	Max Score: 5.0	Min Score: 0.0	loss: 0.00107	ex_p: 0.84	time: 19.40
Ep 237	Score: 2.0	Average Scores: 1.850	Max Score: 5.0	Min Score: 0.0	loss: 0.00113	ex_p: 0.84	time: 21.89
Ep 238	Score: 4.0	Average Scores: 1.870	Max Score: 5.0	Min Score: 0.0	loss: 0.00121	ex_p: 0.84	time: 32.02
Ep 239	Score: 3.0	Average Scores: 1.870	Max Score: 5.0	Min Score: 0.0	loss: 0.00118	ex_p: 0.84	time: 26.51
Ep 240	Score: 2.0	Average Scores: 1.850	Max Score: 5.0	Min Score: 0.0	loss: 0.00123	ex_p: 0.84	time: 21.60
Ep 241	Score: 4.0	Average Scores: 1.8

...

Ep 920	Score: 3.0	Average Scores: 5.080	Max Score: 11.0	Min Score: 1.0	loss: 0.00065	ex_p: 0.41	time: 45.68
Ep 921	Score: 7.0	Average Scores: 5.100	Max Score: 11.0	Min Score: 1.0	loss: 0.00097	ex_p: 0.41	time: 85.02
Ep 922	Score: 8.0	Average Scores: 5.160	Max Score: 11.0	Min Score: 1.0	loss: 0.00194	ex_p: 0.41	time: 80.84
Ep 923	Score: 2.0	Average Scores: 5.140	Max Score: 11.0	Min Score: 1.0	loss: 0.00094	ex_p: 0.41	time: 48.95
Ep 924	Score: 4.0	Average Scores: 5.120	Max Score: 11.0	Min Score: 1.0	loss: 0.00092	ex_p: 0.41	time: 65.73
Ep 925	Score: 2.0	Average Scores: 5.090	Max Score: 11.0	Min Score: 1.0	loss: 0.00213	ex_p: 0.41	time: 49.57
Ep 926	Score: 7.0	Average Scores: 5.100	Max Score: 11.0	Min Score: 1.0	loss: 0.00076	ex_p: 0.41	time: 85.85
Ep 927	Score: 4.0	Average Scores: 5.090	Max Score: 11.0	Min Score: 1.0	loss: 0.00139	ex_p: 0.41	time: 57.26
Ep 928	Score: 11.0	Average Scores: 5.140	Max Score: 11.0	Min Score: 1.0	loss: 0.00144	ex_p: 0.41	time: 92.73
Ep 929	Score: 9.0	Average S

Ep 996	Score: 4.0	Average Scores: 5.520	Max Score: 15.0	Min Score: 1.0	loss: 0.00082	ex_p: 0.38	time: 75.97
Ep 997	Score: 3.0	Average Scores: 5.510	Max Score: 15.0	Min Score: 1.0	loss: 0.00250	ex_p: 0.38	time: 69.71
Ep 998	Score: 15.0	Average Scores: 5.570	Max Score: 15.0	Min Score: 1.0	loss: 0.00155	ex_p: 0.38	time: 159.62
Ep 999	Score: 7.0	Average Scores: 5.590	Max Score: 15.0	Min Score: 1.0	loss: 0.00246	ex_p: 0.38	time: 108.10
Ep 1000	Score: 0.0	Average Scores: 5.550	Max Score: 15.0	Min Score: 0.0	loss: 0.00216	ex_p: 0.38	time: 39.33
Ep 1001	Score: 10.0	Average Scores: 5.570	Max Score: 15.0	Min Score: 0.0	loss: 0.00237	ex_p: 0.38	time: 127.09
Ep 1002	Score: 7.0	Average Scores: 5.560	Max Score: 15.0	Min Score: 0.0	loss: 0.00125	ex_p: 0.38	time: 97.11
Ep 1003	Score: 4.0	Average Scores: 5.580	Max Score: 15.0	Min Score: 0.0	loss: 0.00301	ex_p: 0.38	time: 67.18
Ep 1004	Score: 5.0	Average Scores: 5.570	Max Score: 15.0	Min Score: 0.0	loss: 0.00273	ex_p: 0.37	time: 92.04
Ep 1005	Score: 3.0

Ep 1071	Score: 6.0	Average Scores: 5.830	Max Score: 15.0	Min Score: 0.0	loss: 0.00139	ex_p: 0.35	time: 110.23
Ep 1072	Score: 8.0	Average Scores: 5.830	Max Score: 15.0	Min Score: 0.0	loss: 0.00081	ex_p: 0.34	time: 140.53
Ep 1073	Score: 5.0	Average Scores: 5.860	Max Score: 15.0	Min Score: 0.0	loss: 0.00123	ex_p: 0.34	time: 93.09
Ep 1074	Score: 7.0	Average Scores: 5.900	Max Score: 15.0	Min Score: 0.0	loss: 0.00178	ex_p: 0.34	time: 128.55
Ep 1075	Score: 7.0	Average Scores: 5.930	Max Score: 15.0	Min Score: 0.0	loss: 0.00231	ex_p: 0.34	time: 138.48
Ep 1076	Score: 5.0	Average Scores: 5.910	Max Score: 15.0	Min Score: 0.0	loss: 0.00210	ex_p: 0.34	time: 94.48
Ep 1077	Score: 7.0	Average Scores: 5.910	Max Score: 15.0	Min Score: 0.0	loss: 0.00087	ex_p: 0.34	time: 122.62
Ep 1078	Score: 7.0	Average Scores: 5.910	Max Score: 15.0	Min Score: 0.0	loss: 0.00111	ex_p: 0.34	time: 121.43
Ep 1079	Score: 6.0	Average Scores: 5.950	Max Score: 15.0	Min Score: 0.0	loss: 0.00118	ex_p: 0.34	time: 114.40
Ep 1080	Scor

Ep 1146	Score: 6.0	Average Scores: 5.750	Max Score: 14.0	Min Score: 2.0	loss: 0.00105	ex_p: 0.32	time: 135.21
Ep 1147	Score: 6.0	Average Scores: 5.750	Max Score: 14.0	Min Score: 2.0	loss: 0.00238	ex_p: 0.32	time: 114.30
Ep 1148	Score: 6.0	Average Scores: 5.750	Max Score: 14.0	Min Score: 2.0	loss: 0.00155	ex_p: 0.32	time: 113.94
Ep 1149	Score: 8.0	Average Scores: 5.690	Max Score: 13.0	Min Score: 2.0	loss: 0.00167	ex_p: 0.32	time: 144.36
Ep 1150	Score: 5.0	Average Scores: 5.700	Max Score: 13.0	Min Score: 2.0	loss: 0.00160	ex_p: 0.31	time: 113.59
Ep 1151	Score: 4.0	Average Scores: 5.670	Max Score: 13.0	Min Score: 2.0	loss: 0.00087	ex_p: 0.31	time: 87.76
Ep 1152	Score: 4.0	Average Scores: 5.650	Max Score: 13.0	Min Score: 2.0	loss: 0.00207	ex_p: 0.31	time: 88.47
Ep 1153	Score: 4.0	Average Scores: 5.630	Max Score: 13.0	Min Score: 2.0	loss: 0.00118	ex_p: 0.31	time: 83.35
Ep 1154	Score: 5.0	Average Scores: 5.560	Max Score: 13.0	Min Score: 2.0	loss: 0.00127	ex_p: 0.31	time: 110.00
Ep 1155	Score

Ep 1221	Score: 6.0	Average Scores: 5.750	Max Score: 15.0	Min Score: 1.0	loss: 0.00112	ex_p: 0.29	time: 121.03
Ep 1222	Score: 6.0	Average Scores: 5.790	Max Score: 15.0	Min Score: 1.0	loss: 0.00138	ex_p: 0.29	time: 142.00
Ep 1223	Score: 6.0	Average Scores: 5.780	Max Score: 15.0	Min Score: 1.0	loss: 0.00115	ex_p: 0.29	time: 110.61
Ep 1224	Score: 3.0	Average Scores: 5.720	Max Score: 15.0	Min Score: 1.0	loss: 0.00069	ex_p: 0.29	time: 93.53
Ep 1225	Score: 7.0	Average Scores: 5.690	Max Score: 15.0	Min Score: 1.0	loss: 0.00100	ex_p: 0.29	time: 146.84
Ep 1226	Score: 5.0	Average Scores: 5.700	Max Score: 15.0	Min Score: 1.0	loss: 0.00114	ex_p: 0.29	time: 107.69
Ep 1227	Score: 6.0	Average Scores: 5.700	Max Score: 15.0	Min Score: 1.0	loss: 0.00156	ex_p: 0.29	time: 131.52
Ep 1228	Score: 8.0	Average Scores: 5.750	Max Score: 15.0	Min Score: 1.0	loss: 0.00168	ex_p: 0.29	time: 135.79
Ep 1229	Score: 7.0	Average Scores: 5.730	Max Score: 15.0	Min Score: 1.0	loss: 0.00322	ex_p: 0.29	time: 142.06
Ep 1230	Sco

# Evaluation